In [1]:
import sys
import pandas as pd
import numpy as np
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
from ckiptagger import construct_dictionary
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

host = 4
host = 0

if host == 0:
    path = r'/Users/aron/Documents/GitHub/Perfume/2_NLP'
    dict_file = r'/Users/aron/Documents/GitHub/Perfume/1_Crawler/Resource/dict.xlsx'
    data_file = r'/Users/aron/Documents/GitHub/Perfume/1_Crawler/Resource/data.xls'
    
elif host == 4:
    path = r'/home/rserver/Data_Mining/personal_workspace/yz/Lab/CkipTagger/'
    dict_file = r'/home/rserver/Data_Mining/personal_workspace/yz/Lab/CkipTagger/Resource/dict.xlsx'
    data_file = r'/home/rserver/Data_Mining/personal_workspace/yz/Lab/CkipTagger/Resource/data.xls'

    
# Codebase ......
path_codebase = [r'/Users/aron/Documents/GitHub/Arsenal/',
                 r'/home/aronhack/stock_predict/Function',
                 r'D:\GitHub\Arsenal',
                 r'D:\Data_Mining\Projects\Codebase_YZ',
                 r'/home/jupyter/Arsenal/20220522',
                 path + '/Function']


for i in path_codebase:    
    if i not in sys.path:
        sys.path = [i] + sys.path
    

import codebase_yz as cbyz
    
    
path_resource = path + '/Resource'
path_export = path + '/Export'

## Import Custom Dictionary

In [2]:
dict_df = pd.read_excel(dict_file)    
dict_li = dict_df['word'].tolist()
custom_dict = dict((el, 1) for el in dict_li)
custom_dict = construct_dictionary(custom_dict)

In [3]:
# 2. Load model
# To use GPU:
#    1. Install tensorflow-gpu (see Installation)
#    2. Set CUDA_VISIBLE_DEVICES environment variable, e.g. os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#    3. Set disable_cuda=False, e.g. ws = WS("./data", disable_cuda=False)
# To use CPU:
# https://drive.google.com/drive/folders/105IKCb88evUyLKlLondvDBoh7Dy_I1tm

ws = WS(path_resource + "/data")
pos = POS(path_resource + "/data")
ner = NER(path_resource + "/data")

/Users/aron/opt/anaconda3/lib/python3.8/site-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/Users/aron/opt/anaconda3/lib/python3.8/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:984: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._kernel = self.add_variable(
/Users/aron/opt/anaconda3/lib/python3.8/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:993: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._bias = self.add_variable(
/Users/aron/opt/anaconda3/lib/python3.8/site-packages/ckiptagger/model_pos.py:56: UserWarning: `tf.nn.rnn_cell.LSTMC

## 4. Run the WS-POS-NER pipeline

In [4]:
data = pd.read_excel(data_file)
data = data.dropna(subset=['title'], axis=0)
data = data[~data['title'].str.contains('廣告')]

# data = data[~data['name'].isna()]
# data = data[data.index<=300]

data = data.dropna(subset=['title'], axis=0)
data['title'] = data['title'].str.replace('找相似', '')

In [5]:
sentence_list = data['title'].tolist()

# y = data['name'].tolist()
y = data['brand'].tolist()

In [6]:
# Test Manually
# data = data[data['title'].str.contains(' 萬寶龍星際旅者男性淡香水')]
sentence_list = data['title'].tolist()

In [7]:
word_sentence_list = ws(
    sentence_list,
    # sentence_segmentation = True, # To consider delimiters
    # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
    # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
    coerce_dictionary = custom_dict, # words in this dictionary are forced
)

pos_sentence_list = pos(word_sentence_list)
entity_sentence_list = ner(word_sentence_list, pos_sentence_list)

In [8]:
df = pd.DataFrame()
for i, sentence in enumerate(sentence_list):
    
    new_li = list(zip(word_sentence_list[i],  pos_sentence_list[i]))
    cur_y = y[i]
    
    temp_df = pd.DataFrame(new_li, columns=['word', 'pos'])
    temp_df['sentence'] = i
    temp_df['tag'] = np.where(temp_df['word']==cur_y, '1', '0')
    
    df = df.append(temp_df)
    
df = df[['sentence', 'word', 'pos', 'tag']]
df

,sentence,word,pos,tag
0,0,/,FW,0
1,0,現貨,Na,0
2,0,速出,VCL,0
3,0,/,FW,0
4,0,皇室,Nc,0
...,...,...,...,...
13,2488,已,D,0
14,2488,售出,VC,0
15,2488,1,FW,0
16,2488,臺中市,Nc,0


In [9]:
df[df['tag']=='1']

,sentence,word,pos,tag
7,0,Claus Porto,FW,1
4,1,Claus Porto,FW,1
0,2,Claus Porto,FW,1
5,4,Claus Porto,FW,1
4,6,Claus Porto,FW,1
...,...,...,...,...
6,2470,Penhaligon's,FW,1
5,2472,Penhaligon's,FW,1
6,2478,Penhaligon's,FW,1
2,2481,Penhaligon's,FW,1


In [10]:
# 6. Show Results
def print_word_pos_sentence(word_sentence, pos_sentence):
    assert len(word_sentence) == len(pos_sentence)
    for word, pos in zip(word_sentence, pos_sentence):
        print(f"{word}({pos})", end="\u3000")
    print()
    return


In [11]:
# word_sentence_list[0]
# pos_sentence_list[0]

df[df['tag']=='1']

,sentence,word,pos,tag
7,0,Claus Porto,FW,1
4,1,Claus Porto,FW,1
0,2,Claus Porto,FW,1
5,4,Claus Porto,FW,1
4,6,Claus Porto,FW,1
...,...,...,...,...
6,2470,Penhaligon's,FW,1
5,2472,Penhaligon's,FW,1
6,2478,Penhaligon's,FW,1
2,2481,Penhaligon's,FW,1


In [12]:
for i, sentence in enumerate(sentence_list):
#     print()
    print(f"'{sentence}'")
    print_word_pos_sentence(word_sentence_list[i],  pos_sentence_list[i])
    for entity in sorted(entity_sentence_list[i]):
        print(entity)

'/現貨速出/ 皇室品牌 Claus Porto 葡萄牙人文香氛 水重奏6款高級訂製香水系列 花香 木質 辛香 禮盒$4,680臺北市松山區'
/(FW)　現貨(Na)　速出(VCL)　/ (FW)　皇室(Nc)　品牌(Na)　 (WHITESPACE)　Claus Porto(FW)　 (WHITESPACE)　葡萄牙(Nc)　人文(Na)　香氛(Na)　 (WHITESPACE)　水(Na)　重奏(Na)　6(Neu)　款(Nf)　高級(VH)　訂製(VC)　香水(Na)　系列(Na)　 (WHITESPACE)　花香(Na)　 (WHITESPACE)　木質(Na)　 (WHITESPACE)　辛香(VH)　 (WHITESPACE)　禮盒(Na)　$(FW)　4,680(Neu)　臺北市(Nc)　松山區(Nc)　
(12, 23, 'ORG', 'Claus Porto')
(24, 27, 'GPE', '葡萄牙')
(35, 36, 'CARDINAL', '6')
(63, 66, 'GPE', '臺北市')
'［已開封］Claus Porto 10ml 5入香水組$350'
［(PARENTHESISCATEGORY)　已(D)　開封(VB)　］(PARENTHESISCATEGORY)　Claus Porto(FW)　 (WHITESPACE)　10(Neu)　ml (Nf)　5(Neu)　入(VCL)　香水(Na)　組(Na)　$(FW)　350(Neu)　
(22, 23, 'CARDINAL', '5')
(27, 31, 'MONEY', '$350')
'Claus Porto - BANHO 躺在你懷中 (佛手柑馬鞭草) 白瓷菱紋香氛蠟燭 70g$1,000新北市三重區'
Claus Porto(FW)　 - BANHO (FW)　躺(VA)　在(P)　你(Nh)　懷中(Nc)　 (WHITESPACE)　((PARENTHESISCATEGORY)　佛手柑(Na)　馬鞭草(Na)　)(PARENTHESISCATEGORY)　 (WHITESPACE)　白瓷(Na)　菱紋(Na)　香氛(Na)　蠟燭(Na)　 70(FW)　g(FW)　$(FW)　1,000(Neu)　新北市(Nc)　三重區(Nc)　
(44, 53, 'MONEY'

蝦皮(Na)　優選(Na)　香(Na)　親(VJ)　香愛(Na)　～(FW)　Bvlgari(FW)　 (WHITESPACE)　大吉嶺(Nc)　極緻版(Na)　 Pour Homme Extreme(FW)　$(FW)　1,638(Neu)　 - $(FW)　1,838(Neu)　已(D)　售出(VC)　 154(Neu)　臺北市(Nc)　大同區(Nc)　
(17, 20, 'GPE', '大吉嶺')
(64, 70, 'GPE', '臺北市大同區')
'BVLGARI寶格麗 勁藍 水能量 男性淡香水 30ml 50ml 100ml TESTER『WNP』$960 - $1,840臺北市大同區'
BVLGARI(FW)　寶格麗(Nb)　 (WHITESPACE)　勁(VH)　藍(VH)　 (WHITESPACE)　水(Na)　能量(Na)　 (WHITESPACE)　男性(Na)　淡香水(Na)　 30(FW)　ml (Nf)　50(Neu)　ml (Nf)　100(Neu)　ml TESTER『WNP(FW)　』(PARENTHESISCATEGORY)　$(FW)　960(Neu)　 - (FW)　$(FW)　1,840(Neu)　臺北市(Nc)　大同區(Nc)　
(0, 10, 'PRODUCT', 'BVLGARI寶格麗')
(64, 70, 'GPE', '臺北市大同區')
'2022 寶格麗歡沁玫香女性淡香水5ml典藏瓶$299已售出 3臺南市北區'
2022(Neu)　 (WHITESPACE)　寶格麗(Nb)　歡沁(Nb)　玫香(Na)　女性(Na)　淡香水(Na)　5(Neu)　ml(Nf)　典藏瓶(Na)　$(FW)　299(Neu)　已(D)　售出(VC)　 3(FW)　臺南市(Nc)　北區(Nc)　
(0, 4, 'CARDINAL', '2022')
(5, 8, 'ORG', '寶格麗')
(9, 12, 'PERSON', '沁玫香')
(17, 20, 'QUANTITY', '5ml')
'【超激敗】BVLGARI 寶格麗 魅夜茉香 女性淡香精 50ML /  100ML Jasmin Noir$1,599已售出 2臺北市大同區'
【(PARENTHESISCATEGORY)　超(Dfa)　激敗(VH)　】(PARENTHESIS

BVLGARI(FW)　寶格麗(Nb)　大吉嶺(Nc)　茶(Na)　淡香水(Na)　 5(FW)　ml (Nf)　試香(Na)　$(FW)　180(Neu)　已(D)　售出(VC)　 92(FW)　臺中市(Nc)　南屯區(Nc)　
(11, 13, 'GPE', '吉嶺')
(31, 34, 'DATE', ' 92')
(34, 40, 'GPE', '臺中市南屯區')
'BVLGARI 寶格麗 羅馬之夜 The Roman Night女性淡香精 30ml 50ml 75ml《小平頭香水店》$1,130已售出 2臺北市大同區'
BVLGARI (FW)　寶格麗(Nb)　 (WHITESPACE)　羅馬(Nc)　之(DE)　夜(Nd)　 The Roman (FW)　Night(FW)　女性(Na)　淡香精(Na)　 (WHITESPACE)　30(Neu)　ml (Nf)　50(Neu)　ml (Nf)　75(Neu)　ml(Nf)　《(PARENTHESISCATEGORY)　小平頭(Na)　香水(Na)　店(Nc)　》(PARENTHESISCATEGORY)　$(FW)　1,130(Neu)　已(D)　售出(VC)　 2(FW)　臺北市(Nc)　大同區(Nc)　
(0, 11, 'ORG', 'BVLGARI 寶格麗')
(53, 59, 'ORG', '小平頭香水店')
(60, 66, 'CARDINAL', '$1,130')
(71, 77, 'GPE', '臺北市大同區')
'蝦皮優選BVLGARI 寶格麗 綠茶/白茶中性古龍水10ml/噴式小香水 香水筆 任選一款$390 - $750已售出 43新北市林口區'
蝦皮(Na)　優選(Na)　BVLGARI (FW)　寶格麗(Nb)　 (WHITESPACE)　綠茶(Na)　/(FW)　白茶(Na)　中性(VH)　古龍水(Na)　10(Neu)　ml/(FW)　噴式(Na)　小(VH)　香水(Na)　 (WHITESPACE)　香水(Na)　筆(Na)　 (WHITESPACE)　任選(VC)　一(Neu)　款(Nf)　$(FW)　390(Neu)　 - $(FW)　750(Neu)　已(D)　售出(VC)　 43(Neu)　新北市(Nc)　林口區(Nc)　
(4, 15, 'PRODUCT', '

5.5(Neu)　折(Nf)　我(Nh)　的(DE)　印記(Na)　淡香水(Na)　  (WHITESPACE)　淡香精(Na)　 MON (FW)　GUERLAIN(FW)　 (WHITESPACE)　針管(Na)　  5(Cbb)　ml (Nf)　花草(Na)　水(Na)　語(Na)　 (WHITESPACE)　針管(Na)　香水(Na)　【(PARENTHESISCATEGORY)　GUERLAIN(FW)　 (WHITESPACE)　嬌蘭(Nb)　】(PARENTHESISCATEGORY)　小凱(Nb)　美妝(Na)　10(Neu)　 (WHITESPACE)　件(Nf)　 (WHITESPACE)　9.7(Neu)　折(Nf)　$(FW)　80(Neu)　 - $(FW)　280(Neu)　已(D)　售出(VC)　 217(Neu)　新北市(Nc)　樹林區(Nc)　
(64, 66, 'CARDINAL', '10')
(73, 76, 'MONEY', '$80')
'愛妮❤️GUERLAIN 嬌蘭 花草水語 淡香水 野漾叢花 /陽光甜橘/沐光沁檸/暖心甜薑/日光澄香/蜜漾晨光125ml$1,980高雄市路竹區'
愛妮❤️(Nb)　GUERLAIN(FW)　 (WHITESPACE)　嬌蘭(Nb)　 (WHITESPACE)　花草(Na)　水(Na)　語(Na)　 (WHITESPACE)　淡香水(Na)　 (WHITESPACE)　野漾(Nd)　叢花(Na)　 /(FW)　陽光(Na)　甜橘(Na)　/(FW)　沐光(Na)　沁檸(Na)　/(FW)　暖心(Na)　甜(VH)　薑(Na)　/(FW)　日光(Na)　澄香(Na)　/(FW)　蜜漾(Nb)　晨光(Na)　125(Neu)　ml(Nf)　$(FW)　1,980(Neu)　高雄市(Nc)　路竹區(Nc)　
(13, 15, 'PRODUCT', '嬌蘭')
(66, 72, 'GPE', '高雄市路竹區')
'Guerlain嬌蘭 La Petite Robe Noire 小黑裙 精粹V領洋裝淡香精 50ML《小平頭香水店》$2,199已售出 1臺北市大同區'
Guerlain(FW)　嬌蘭(Nb)　 La (FW)　Petite Robe (FW)　Noire (FW)　小黑裙(Na)　 

專櫃(Nc)　正品(Na)　🐝(FW)　GUERLAIN(FW)　 (WHITESPACE)　嬌蘭(Nb)　🐝(Na)　❤️(Na)　我(Nh)　的(DE)　印記(Na)　玫瑰(Na)　淡香精(Na)　、(PAUSECATEGORY)　花草(Na)　水(Na)　語(Na)　夏恣(VH)　紅榴(Na)　淡香水(Na)　 (WHITESPACE)　，(COMMACATEGORY)　1(Neu)　ml(Nf)　針管(Na)　香水(Na)　❤(FW)　️(Na)　$(FW)　40(Neu)　臺南市(Nc)　仁德區(Nc)　
(42, 45, 'QUANTITY', '1ml')
(51, 54, 'MONEY', '$40')
(54, 60, 'GPE', '臺南市仁德區')
'*人人美*Guerlain小黑裙搖滾皮衣玫瑰淡香水針管0.7ml期限2023.01$49已售出 2'
*(FW)　人人(Na)　美(VH)　*(FW)　Guerlain(FW)　小(VH)　黑裙(Na)　搖滾(Na)　皮衣(Na)　玫瑰(Na)　淡香水(Na)　針管(Na)　0.7(Neu)　ml(Nf)　期限(Na)　2023.01(Neu)　$(FW)　49(Neu)　已(D)　售出(VC)　 2(FW)　
(5, 13, 'PRODUCT', 'Guerlain')
(27, 32, 'QUANTITY', '0.7ml')
(34, 44, 'MONEY', '2023.01$49')
'正貨 嬌蘭香水 GUERLAIN 香水 午夜飛行 女士香水 木質香調 夜航Vol de Nuit 100ML g6B1 $1,599'
正貨(Na)　 (WHITESPACE)　嬌蘭(Nb)　香水(Na)　 (WHITESPACE)　GUERLAIN(FW)　 (WHITESPACE)　香水(Na)　 (WHITESPACE)　午夜(Nd)　飛行(VA)　 (WHITESPACE)　女士(Na)　香水(Na)　 (WHITESPACE)　木質香(Na)　調(Na)　 (WHITESPACE)　夜航(VA)　Vol de Nuit (FW)　100(Neu)　ML g6(FW)　B1(Neu)　 $(FW)　1,599(Neu)　
(3, 5, 'PRODUCT', '嬌蘭')
(20, 22, 'TIME

...

蝦皮(Na)　優選(Na)　☆MOMO(FW)　小屋(Na)　☆ (FW)　Salvatore Ferragamo(FW)　 (WHITESPACE)　Free (FW)　Time (FW)　非凡(VH)　之(DE)　旅(Na)　 (WHITESPACE)　男香(Na)　 100(FW)　ml TESTER$(FW)　990(Neu)　已(D)　售出(VC)　 15(FW)　臺北市(Nc)　萬華區(Nc)　
(71, 73, 'DATE', '15')
(73, 79, 'GPE', '臺北市萬華區')
'【超激敗】Salvatore Ferragamo 繽紛奇境 女性淡香水 100ML Heaven$729臺北市大同區'
【(PARENTHESISCATEGORY)　超(Dfa)　激敗(VH)　】(PARENTHESISCATEGORY)　Salvatore Ferragamo(FW)　 (WHITESPACE)　繽紛(VH)　奇境(Na)　 (WHITESPACE)　女性(Na)　淡香水(Na)　 100(FW)　ML Heaven(FW)　$(FW)　729(Neu)　臺北市(Nc)　大同區(Nc)　
(5, 24, 'ORG', 'Salvatore Ferragamo')
(52, 58, 'GPE', '臺北市大同區')
'Salvatore Ferragamo Uomo  峰範男性淡香水  試香$40 - $110已售出 15桃園市龜山區'
Salvatore Ferragamo(FW)　 (WHITESPACE)　Uomo  (FW)　峰範(Nc)　男性(Na)　淡香水(Na)　  (WHITESPACE)　試香(Na)　$(FW)　40(Neu)　 - $(FW)　110(Neu)　已(D)　售出(VC)　 15(FW)　桃園市(Nc)　龜山區(Nc)　
(30, 33, 'PERSON', '淡香水')
(53, 59, 'GPE', '桃園市龜山區')
'【魔力】Salvatore Ferragamo 甜心魔力 女性 淡香水 100ml/TESTER$550臺北市中正區'
【(PARENTHESISCATEGORY)　魔力(Na)　】(PARENTHESISCATEGORY)　Salvatore Ferragamo(FW)　 (WHITESPA

「(PARENTHESISCATEGORY)　KABOUR (FW)　來自(VJ)　杜拜(Nc)　的(DE)　高品質(Na)　香水(Na)　」(PARENTHESISCATEGORY)　 (WHITESPACE)　Versace(FW)　 Bright Crystal (FW)　相戀(VH)　水(Na)　晶(Na)　 (WHITESPACE)　同(Nes)　香(Na)　$(FW)　600(Neu)　已(D)　售出(VC)　 19(FW)　彰化縣(Nc)　彰化市(Nc)　
(10, 12, 'GPE', '杜拜')
(50, 54, 'MONEY', '$600')
(57, 60, 'DATE', ' 19')
(60, 66, 'GPE', '彰化縣彰化市')
'✰YENGEE✰Versace Eros 凡賽斯艾諾斯情緣 愛神 男性淡香水30ML/100ml/50ML$870 - $1,500已售出 18臺北市大同區'
✰YENGEE✰(FW)　Versace(FW)　 Eros (FW)　凡賽斯(Nb)　艾諾斯(Nb)　情緣(Na)　 (WHITESPACE)　愛神(Nb)　 (WHITESPACE)　男性(Na)　淡香水(Na)　30(Neu)　ML/(FW)　100(Neu)　ml/50(FW)　ML(FW)　$(FW)　870(Neu)　 - $(FW)　1,500(Neu)　已(D)　售出(VC)　 18(FW)　臺北市(Nc)　大同區(Nc)　
(70, 72, 'DATE', '18')
(72, 78, 'GPE', '臺北市大同區')
'✰YENGEE✰VERSACE Red Jeans 凡賽斯 紅可樂 女性淡香水 75ml TESTER$539已售出 5臺北市大同區'
✰YENGEE✰VERSACE Red Jeans (FW)　凡賽斯(Nb)　 (WHITESPACE)　紅可樂(Nb)　 (WHITESPACE)　女性(Na)　淡香水(Na)　 75(FW)　ml TESTER(FW)　$(FW)　539(Neu)　已(D)　售出(VC)　 5(FW)　臺北市(Nc)　大同區(Nc)　
(26, 29, 'PRODUCT', '凡賽斯')
(60, 66, 'GPE', '臺北市大同區')
'🔥現貨🔥 Versace Pour Homme D

Moschino (FW)　Fresh (FW)　Couture (FW)　小清新(Na)　/(FW)　小(VH)　粉紅(VH)　/(FW)　亮(VH)　金金(Na)　女性(Na)　淡香水(Na)　 (WHITESPACE)　100(Neu)　ml TESTER (FW)　包裝(VC)　『(PARENTHESISCATEGORY)　WNP(FW)　』(PARENTHESISCATEGORY)　$(FW)　940(Neu)　已(D)　售出(VC)　 145(Neu)　臺北市(Nc)　大同區(Nc)　
(40, 43, 'CARDINAL', '100')
(56, 59, 'GPE', 'WNP')
(60, 64, 'PRODUCT', '$940')
(71, 77, 'GPE', '臺北市大同區')
'✰YENGEE✰MOSCHINO FRESH COUTURE 小清新 / 小粉紅/ 亮金金 清潔劑 女性淡香水 5ml$169已售出 1,730臺北市大同區'
✰YENGEE✰(FW)　MOSCHINO(FW)　 FRESH COUTURE (FW)　小清新(Na)　 / (FW)　小(VH)　粉紅(VH)　/ (FW)　亮金金(Na)　 (WHITESPACE)　清潔劑(Na)　 (WHITESPACE)　女性(Na)　淡香水(Na)　 5(FW)　ml(Nf)　$(FW)　169(Neu)　已(D)　售出(VC)　 1,730(Neu)　臺北市(Nc)　大同區(Nc)　
(56, 63, 'QUANTITY', '5ml$169')
(69, 72, 'PERCENT', '730')
(72, 78, 'GPE', '臺北市大同區')
'愛妮❤️Moschino Toy2 熊芯未泯2 toy 2女性淡香精30ml/50ml/100ml /【TESTER】♫$699 - $1,280已售出 196高雄市路竹區'
愛妮❤️(Nb)　Moschino (FW)　Toy(FW)　2 (Neu)　熊芯(Nb)　未(D)　泯(VC)　2 toy 2(FW)　女性(Na)　淡香精(Na)　30(Neu)　ml/(FW)　50(Neu)　ml/(FW)　100(Neu)　ml /(FW)　【(PARENTHESISCATEGORY)　TESTER(FW)　】(PARENTH

♛GEM(FW)　小舖(Nc)　♛(FW)　【(PARENTHESISCATEGORY)　GATSBY(FW)　】(PARENTHESISCATEGORY)　男性(Na)　古龍(Nb)　香水(Na)　 125(Neu)　ml  (FW)　無懈可擊_(VH)　無盡(VH)　的(DE)　愛_(VH)　自由(VH)　氣息(Na)　 (WHITESPACE)　香水(Na)　/(FW)　古龍水(Na)　滿額(VH)　贈(VD)　$(FW)　85(Neu)　已(D)　售出(VC)　 570(FW)　新北市(Nc)　中和區(Nc)　
(53, 56, 'MONEY', '$85')
(59, 69, 'GPE', ' 570新北市中和區')
'蝦皮優選GATSBY 男性淡香水 飛揚恣意/鉑金王者/極致湛藍/晨霧之光/曠野能量 50ml 多款供選 【小元寶】超取$159已售出 28臺中市南區'
蝦皮(Na)　優選(Na)　GATSBY(FW)　 (WHITESPACE)　男性(Na)　淡香水(Na)　 (WHITESPACE)　飛揚(VA)　恣意(D)　/(FW)　鉑金王(Na)　者(Na)　/(FW)　極致(Na)　湛藍(VH)　/(FW)　晨霧(Na)　之(DE)　光(Na)　/(FW)　曠野(Na)　能量(Na)　 (WHITESPACE)　50(Neu)　ml (Nf)　多(Neqb)　款(Nf)　供選(VC)　 (WHITESPACE)　【(PARENTHESISCATEGORY)　小元寶(Na)　】(PARENTHESISCATEGORY)　超取(VJ)　$(FW)　159(Neu)　已(D)　售出(VC)　 28(FW)　臺中市(Nc)　南區(Nc)　
(42, 44, 'CARDINAL', '50')
(59, 63, 'MONEY', '$159')
(67, 69, 'DATE', '28')
(69, 72, 'GPE', '臺中市')
'現貨【 GATSBY】男士古龍水 男香水 7款125ml 體香 男香滿額贈$85已售出 236高雄市新興區'
現貨(Na)　【(PARENTHESISCATEGORY)　 (WHITESPACE)　GATSBY(FW)　】(PARENTHESISCATEGORY)　男士(Na)　古龍水(Na)　 (WHITESPACE)　

DUNHILL fragrance (FW)　香水(Na)　、(PAUSECATEGORY)　化妝品(Na)　、(PAUSECATEGORY)　皮包(Na)　、(PAUSECATEGORY)　眼鏡(Na)　、(PAUSECATEGORY)　大哥大(Na)　萬用(A)　手(Na)　拿包(Na)　$(FW)　1,800(Neu)　臺中市(Nc)　豐原區(Nc)　
(0, 18, 'ORG', 'DUNHILL fragrance ')
(39, 45, 'CARDINAL', '$1,800')
(45, 51, 'GPE', '臺中市豐原區')
'Dunhill世紀紳藍男性淡香精30ml(2024/10)$599'
Dunhill(FW)　世紀(Na)　紳藍(VH)　男性(Na)　淡香精(Na)　30(Neu)　ml(Nf)　((PARENTHESISCATEGORY)　2024(Neu)　/(FW)　10(Neu)　)(PARENTHESISCATEGORY)　$(FW)　599(Neu)　
(9, 11, 'PERSON', '紳藍')
(16, 20, 'QUANTITY', '30ml')
'dunhill 登喜路 ICON RACING 極速 男性淡香精 50ml$1,490臺北市萬華區'
dunhill (FW)　登喜路(Nb)　 ICON RACING (FW)　極速(Na)　 (WHITESPACE)　男性(Na)　淡香精(Na)　 50(FW)　ml(Nf)　$(FW)　1,490(Neu)　臺北市(Nc)　萬華區(Nc)　
(0, 8, 'PERSON', 'dunhill ')
(8, 11, 'PERSON', '登喜路')
(43, 46, 'GPE', '臺北市')
(46, 49, 'GPE', '萬華區')
'Dunhill 中心點男性淡香水禮盒$3,500臺中市西屯區'
Dunhill(FW)　 (WHITESPACE)　中心點(Nc)　男性(Na)　淡香水(Na)　禮盒(Na)　$(FW)　3,500(Neu)　臺中市(Nc)　西屯區(Nc)　
(0, 7, 'PRODUCT', 'Dunhill')
(18, 24, 'MONEY', '$3,500')
(24, 30, 'GPE', '臺中市西屯區')
'Dunhill 登喜路

## Feature Extraction

In [13]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['word'].values.tolist(), 
                                                           s['pos'].values.tolist(), 
                                                           s['tag'].values.tolist())]
        self.grouped = self.data.groupby('sentence').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None

        
getter = SentenceGetter(df)
sentences = getter.sentences

In [14]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
        
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]
def sent2labels(sent):
    return [label for token, postag, label in sent]
def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [15]:
### Split train and test sets
X = [sent2features(s) for s in sentences]
new_y = [sent2labels(s) for s in sentences]
X_train, X_test, y_train, y_test = train_test_split(X, new_y, test_size=0.33, random_state=0)

### Train a CRF model

In [16]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

In [17]:
trainer.set_params({
#     'c1': 1.0,   # coefficient for L1 penalty
#     'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

In [18]:
trainer.params()

['feature.minfreq',
 'feature.possible_states',
 'feature.possible_transitions',
 'c1',
 'c2',
 'max_iterations',
 'num_memories',
 'epsilon',
 'period',
 'delta',
 'linesearch',
 'max_linesearch']

In [19]:
trainer.train(path_export + '/ckip_tagger_model.crfsuite')

In [20]:
trainer.logparser.last_iteration

{'num': 50,
 'scores': {},
 'loss': 356.061842,
 'feature_norm': 13.64818,
 'error_norm': 1.329007,
 'active_features': 19291,
 'linesearch_trials': 1,
 'linesearch_step': 1.0,
 'time': 0.013}

In [21]:
print(len(trainer.logparser.iterations), trainer.logparser.iterations[-1])

50 {'num': 50, 'scores': {}, 'loss': 356.061842, 'feature_norm': 13.64818, 'error_norm': 1.329007, 'active_features': 19291, 'linesearch_trials': 1, 'linesearch_step': 1.0, 'time': 0.013}


## Make predictions

To use the trained model, create pycrfsuite.Tagger, open the model and use "tag" method:

In [ ]:
tagger = pycrfsuite.Tagger()
tagger.open(path_export + '/ckip_tagger_model.crfsuite')

Let's tag a sentence to see how it works:

In [ ]:
y_pred = []
for i in range(len(y_test)):
    new_pred = tagger.tag(X_test[i])
    y_pred.append(new_pred)
#     example_sent = X_test[i]
#     print("Predicted:", ' '.join(tagger.tag(X_test[i])))    

In [ ]:
y_pred = cbyz.li_flatten(y_pred)
y_test = cbyz.li_flatten(y_test)

In [ ]:
# len(y_pred)
len(X_test)

In [ ]:
y_test[i for i in y_test]

In [ ]:
print(classification_report(y_pred=y_pred, y_true=y_test))
# print(classification_report(y_pred=y_pred, y_true=y_test, labels=new_classes))

## Let's check what classifier learned

In [ ]:
from collections import Counter
info = tagger.info()

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(info.transitions).most_common(15))

print("\nTop unlikely transitions:")
print_transitions(Counter(info.transitions).most_common()[-15:])

We can see that, for example, it is very likely that the beginning of an organization name (B-ORG) will be followed by a token inside organization name (I-ORG), but transitions to I-ORG from tokens with other labels are penalized. Also note I-PER -> B-LOC transition: a positive weight means that model thinks that a person name is often followed by a location.

Check the state features:

In [ ]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-6s %s" % (weight, label, attr))    

print("Top positive:")
print_state_features(Counter(info.state_features).most_common(20))

print("\nTop negative:")
print_state_features(Counter(info.state_features).most_common()[-20:])

## Word2Vec

In [22]:
import gensim
from gensim import matutils
from gensim.models.word2vec import Word2Vec

model = gensim.models.KeyedVectors.load_word2vec_format(
        path_resource + '/tmunlp_1.6B_WB_50dim_2020v1.bin.gz', 
        unicode_errors='ignore',
        binary=True
    )

In [25]:
# model.vocab

In [26]:
# for i in [0]:
result = pd.DataFrame()

for i in range(len(word_sentence_list)):

    word_vec = {}
    for w in word_sentence_list[i]:
        try:
            word_vec[w] = model.get_vector(w)
        except:
            pass

    # Create DF
    keys = list(word_vec.keys())
    vec_df = pd.DataFrame({'word':keys,
                          'index':range(len(keys))})

    vec_df = cbyz.df_cross_join(vec_df, vec_df)
    vec_df = vec_df[vec_df['index_x']<vec_df['index_y']] \
            .reset_index(drop=True) 
    
    for j in range(len(vec_df)):
        vec1 = model.get_vector(vec_df.loc[j, 'word_x'])
        vec2 = model.get_vector(vec_df.loc[j, 'word_y'])
        similarity = np.dot(matutils.unitvec(vec1), matutils.unitvec(vec2))
        vec_df.loc[j, 'similarity'] = similarity

    result = result.append(vec_df)
    
    if i % 100 == 0:
        print(i)
    
result = result.reset_index(drop=True)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400


In [28]:
result[result['similarity']>0.5]

,word_x,index_x,word_y,index_y,similarity
74,皇室,3,高級,12,0.544621
76,皇室,3,香水,14,0.538862
95,品牌,4,香水,14,0.523832
96,品牌,4,系列,15,0.525878
146,香氛,7,香水,14,0.855557
...,...,...,...,...,...
312678,1.5,7,170,10,0.777990
312703,臺東縣,13,卑南鄉,14,0.601798
312762,30,6,999,8,0.546497
312767,$,7,999,8,0.518172


In [ ]:
type(model)

In [ ]:
# model.vectors
# model.vocab

In [ ]:
len(model.vocab)

## Worklist

In [ ]:
# - 跟淡香水之間的距離
#   和男性、女性、中性的距離
# - 如果title中同時出現「淡香水」、「淡香精」就排除